In [1]:
import pandas as pd
import json
import os
import numpy as np
import re #regex 

In [2]:
dfs = []
for r, d, f in os.walk('input'):
    for file in f:
        if 'withheldtweets.json' in file or 'plus_one_control.json' in file:  # alt: if ‘control' in file:
            dfs.append(pd.read_json('%s/%s' % (r, file), lines=True))

df_cen = pd.concat(dfs)
#df_cen = df_cen.dropna(subset=['withheld_in_countries'])

In [3]:
worthKeeping = ["text", "truncated", "user",
                "withheld_in_countries", "entities", "lang",
                "possibly_sensitive", "extended_tweet"]
df_cen = df_cen[worthKeeping]

In [4]:
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,NaN,NaN
1,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,NaN,NaN
2,RT @sunochanda12: جسکو ہمارے ملک کے لبدلز معصو...,False,"{'id': 421049995, 'id_str': '421049995', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,NaN,NaN
3,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
4,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,NaN,NaN
...,...,...,...,...,...,...,...,...
178,RT @Mira_n420: 私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなん...,False,"{'id': 3232626306, 'id_str': '3232626306', 'na...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,NaN,NaN
179,RT @minombreeschris: todo el mundo manda foto ...,False,"{'id': 1267532670562701312, 'id_str': '1267532...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",es,0.0,NaN
180,@iiSpez درر,False,"{'id': 353999477, 'id_str': '353999477', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ar,NaN,NaN
181,ja participo des do ano passado,False,"{'id': 1092518124455448576, 'id_str': '1092518...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",pt,NaN,NaN


In [5]:
#replace the NaN with coherent values to make further processing easier
df_cen['possibly_sensitive'] = df_cen['possibly_sensitive'].fillna(0.0)
df_cen

,text,truncated,user,withheld_in_countries,entities,lang,possibly_sensitive,extended_tweet
0,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,0.0,NaN
1,RT @ZaidZamanHamid: کرونا ایسے تباہی پھیلا رہا...,False,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,0.0,NaN
2,RT @sunochanda12: جسکو ہمارے ملک کے لبدلز معصو...,False,"{'id': 421049995, 'id_str': '421049995', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",ur,0.0,NaN
3,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
4,"RT @ZaidZamanHamid: If it is blood clotting, w...",False,"{'id': 300795161, 'id_str': '300795161', 'name...",[IN],"{'hashtags': [], 'urls': [], 'user_mentions': ...",en,0.0,NaN
...,...,...,...,...,...,...,...,...
178,RT @Mira_n420: 私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなん...,False,"{'id': 3232626306, 'id_str': '3232626306', 'na...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ja,0.0,NaN
179,RT @minombreeschris: todo el mundo manda foto ...,False,"{'id': 1267532670562701312, 'id_str': '1267532...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",es,0.0,NaN
180,@iiSpez درر,False,"{'id': 353999477, 'id_str': '353999477', 'name...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",ar,0.0,NaN
181,ja participo des do ano passado,False,"{'id': 1092518124455448576, 'id_str': '1092518...",NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",pt,0.0,NaN


In [6]:
#recover the full text for truncated tweets

dfRaw = df_cen.values
for line in dfRaw:
    if not pd.isna(line[-1]):
        line[0] = line[-1]["full_text"]
        
    #remove urls from tweets
    #they are shortened anyway so we can't make use of them
    line[0] = re.sub(r'http\S+', '', line[0])
    
    #flatten retweets
    line[0] = re.sub(r'RT @\S+:', '', line[0])

dfRaw = np.delete(dfRaw, len(worthKeeping)-1, axis=1) #remove "extended_tweet"
worthKeeping.remove("extended_tweet")

dfRaw = np.delete(dfRaw, 1, axis=1) #remove "truncated"
worthKeeping.remove("truncated")

In [7]:
#extract hashtags seperately

for line in dfRaw:
    line[3] = [x["text"] for x in line[3]["hashtags"]]
worthKeeping[3] = "hashtags"

In [8]:
#create a feature for user-verified and user-followers_count

print(dfRaw[0][1].keys())
verified = [line[1]["verified"] for line in dfRaw]
followers = [line[1]["followers_count"] for line in dfRaw]

#for the location, Rebekah suggested to only spot the country name and discard the rest
listOfCountries = ['Afghanistan', 'Aland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia, Plurinational State of', 'Bonaire, Sint Eustatius and Saba', 'Bosnia and Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo', 'Congo, The Democratic Republic of the', 'Cook Islands', 'Costa Rica', "Côte d'Ivoire", 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Ethiopia', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See (Vatican City State)', 'Honduras', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran, Islamic Republic of', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', "Korea, Democratic People's Republic of", 'Korea, Republic of', 'Kuwait', 'Kyrgyzstan', "Lao People's Democratic Republic", 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao', 'Macedonia, Republic of', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia, Federated States of', 'Moldova, Republic of', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territory, Occupied', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russian Federation', 'Rwanda', 'Saint Barthélemy', 'Saint Helena, Ascension and Tristan da Cunha', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Martin (French part)', 'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines', 'Samoa', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten (Dutch part)', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia and the South Sandwich Islands', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'South Sudan', 'Svalbard and Jan Mayen', 'Swaziland', 'Sweden', 'Switzerland', 'Syrian Arab Republic', 'Taiwan, Province of China', 'Tajikistan', 'Tanzania, United Republic of', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks and Caicos Islands', 'Tuvalu', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'United States Minor Outlying Islands', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Venezuela, Bolivarian Republic of', 'Viet Nam', 'Virgin Islands, British', 'Virgin Islands, U.S.', 'Wallis and Futuna', 'Yemen', 'Zambia', 'Zimbabwe']
def findCountry(x):
    for country in listOfCountries:
        if x and country in x:
            return country
    return None

location = [findCountry(line[1]["location"]) for line in dfRaw]

dfRaw = np.c_[dfRaw, verified, followers, location]
worthKeeping += ["verified_account", "followers_count", "location"]

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'url', 'description', 'translator_type', 'protected', 'verified', 'followers_count', 'friends_count', 'listed_count', 'favourites_count', 'statuses_count', 'created_at', 'utc_offset', 'time_zone', 'geo_enabled', 'lang', 'contributors_enabled', 'is_translator', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'profile_image_url', 'profile_image_url_https', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications'])


In [9]:
#binary feature for whether the tweet has been withheld anywhere

withheld = []
for line in dfRaw:
    if not isinstance(line[2], list):
        line[2] = []
    withheld.append(len(line[2]) != 0)
        
dfRaw = np.c_[dfRaw, withheld]
worthKeeping += ["withheld_anywhere"]

In [10]:
#sentiment analysis
#https://www.analyticsvidhya.com/blog/2022/07/sentiment-analysis-using-python/? with VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

sentiment = SentimentIntensityAnalyzer()
#we made the assumption that sentiment analysis for this analyzer only works for english
res = np.array([[x for x in sentiment.polarity_scores(line[0]).values()] if line[4] == "en" else [0.0, 0.0, 0.0, 0.0] for line in dfRaw])

dfRaw = np.c_[dfRaw, res]
worthKeeping += ["neg", "neu", "pos", "compound"]

In [11]:
#popularity feature:
#build a score based on the values of followers_count, favourites_count, statuses_count
#compute a score from 0 to 1 for each, with (x - min)/(max - min), then comptute the average of these scores 

followers_count = np.array([line[1]["followers_count"] for line in dfRaw])
favourites_count = np.array([line[1]["favourites_count"] for line in dfRaw])
statuses_count = np.array([line[1]["statuses_count"] for line in dfRaw])

def normalize(array):
    return (array - np.min(array)) / (np.max(array) - np.min(array))

score = (1/3) * (normalize(followers_count) + normalize(favourites_count) + normalize(statuses_count))
dfRaw = np.c_[dfRaw, score]
worthKeeping += ["popularity_score"]

In [12]:
#reassemble the data in a pandas dataframe
df_cen = pd.DataFrame(dfRaw, columns = worthKeeping)
df_cen

,text,user,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,کرونا ایسے تباہی پھیلا رہا ہے جیسے 22 سال کی ...,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],[],ur,0.0,False,4,None,True,0.0,0.0,0.0,0.0,0.000031
1,کرونا ایسے تباہی پھیلا رہا ہے جیسے 22 سال کی ...,"{'id': 1124646527895576579, 'id_str': '1124646...",[IN],[],ur,0.0,False,4,None,True,0.0,0.0,0.0,0.0,0.000031
2,جسکو ہمارے ملک کے لبدلز معصوم بنا کر پیش کررہ...,"{'id': 421049995, 'id_str': '421049995', 'name...",[IN],[],ur,0.0,False,371,Pakistan,True,0.0,0.0,0.0,0.0,0.005671
3,"If it is blood clotting, wont it clot the blo...","{'id': 300795161, 'id_str': '300795161', 'name...",[IN],[],en,0.0,False,2471,None,True,0.0,0.916,0.084,0.3384,0.015019
4,"If it is blood clotting, wont it clot the blo...","{'id': 300795161, 'id_str': '300795161', 'name...",[IN],[],en,0.0,False,2471,None,True,0.0,0.916,0.084,0.3384,0.015019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70883,私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなんと！新プロジェクトを立ち上げ...,"{'id': 3232626306, 'id_str': '3232626306', 'na...",[],[],ja,0.0,False,316,None,False,0.0,0.0,0.0,0.0,0.019546
70884,"todo el mundo manda foto de pinta el 31, mejo...","{'id': 1267532670562701312, 'id_str': '1267532...",[],[],es,0.0,False,178,None,False,0.0,0.0,0.0,0.0,0.0006
70885,@iiSpez درر,"{'id': 353999477, 'id_str': '353999477', 'name...",[],[],ar,0.0,False,13770,None,False,0.0,0.0,0.0,0.0,0.002848
70886,ja participo des do ano passado,"{'id': 1092518124455448576, 'id_str': '1092518...",[],[],pt,0.0,False,404,None,False,0.0,0.0,0.0,0.0,0.001821


In [13]:
cleanCols = filter(lambda x: x != "user", worthKeeping)
df_clean = df_cen[cleanCols]

In [14]:
df_clean

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,کرونا ایسے تباہی پھیلا رہا ہے جیسے 22 سال کی ...,[IN],[],ur,0.0,False,4,None,True,0.0,0.0,0.0,0.0,0.000031
1,کرونا ایسے تباہی پھیلا رہا ہے جیسے 22 سال کی ...,[IN],[],ur,0.0,False,4,None,True,0.0,0.0,0.0,0.0,0.000031
2,جسکو ہمارے ملک کے لبدلز معصوم بنا کر پیش کررہ...,[IN],[],ur,0.0,False,371,Pakistan,True,0.0,0.0,0.0,0.0,0.005671
3,"If it is blood clotting, wont it clot the blo...",[IN],[],en,0.0,False,2471,None,True,0.0,0.916,0.084,0.3384,0.015019
4,"If it is blood clotting, wont it clot the blo...",[IN],[],en,0.0,False,2471,None,True,0.0,0.916,0.084,0.3384,0.015019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70883,私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなんと！新プロジェクトを立ち上げ...,[],[],ja,0.0,False,316,None,False,0.0,0.0,0.0,0.0,0.019546
70884,"todo el mundo manda foto de pinta el 31, mejo...",[],[],es,0.0,False,178,None,False,0.0,0.0,0.0,0.0,0.0006
70885,@iiSpez درر,[],[],ar,0.0,False,13770,None,False,0.0,0.0,0.0,0.0,0.002848
70886,ja participo des do ano passado,[],[],pt,0.0,False,404,None,False,0.0,0.0,0.0,0.0,0.001821


In [15]:
df_clean["withheld_anywhere"].describe()

count     70888
unique        2
top        True
freq      41733
Name: withheld_anywhere, dtype: object

In [16]:
df_clean

,text,withheld_in_countries,hashtags,lang,possibly_sensitive,verified_account,followers_count,location,withheld_anywhere,neg,neu,pos,compound,popularity_score
0,کرونا ایسے تباہی پھیلا رہا ہے جیسے 22 سال کی ...,[IN],[],ur,0.0,False,4,None,True,0.0,0.0,0.0,0.0,0.000031
1,کرونا ایسے تباہی پھیلا رہا ہے جیسے 22 سال کی ...,[IN],[],ur,0.0,False,4,None,True,0.0,0.0,0.0,0.0,0.000031
2,جسکو ہمارے ملک کے لبدلز معصوم بنا کر پیش کررہ...,[IN],[],ur,0.0,False,371,Pakistan,True,0.0,0.0,0.0,0.0,0.005671
3,"If it is blood clotting, wont it clot the blo...",[IN],[],en,0.0,False,2471,None,True,0.0,0.916,0.084,0.3384,0.015019
4,"If it is blood clotting, wont it clot the blo...",[IN],[],en,0.0,False,2471,None,True,0.0,0.916,0.084,0.3384,0.015019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70883,私に殺陣は楽しいんだよ🎶\nと教えてくれた方！先生が！\nなんと！新プロジェクトを立ち上げ...,[],[],ja,0.0,False,316,None,False,0.0,0.0,0.0,0.0,0.019546
70884,"todo el mundo manda foto de pinta el 31, mejo...",[],[],es,0.0,False,178,None,False,0.0,0.0,0.0,0.0,0.0006
70885,@iiSpez درر,[],[],ar,0.0,False,13770,None,False,0.0,0.0,0.0,0.0,0.002848
70886,ja participo des do ano passado,[],[],pt,0.0,False,404,None,False,0.0,0.0,0.0,0.0,0.001821


In [17]:
df_clean["neg"].describe()

count     70888.0
unique      594.0
top           0.0
freq      60553.0
Name: neg, dtype: float64

In [18]:
df_clean["popularity_score"].describe()

count     70888.0
unique    56870.0
top           0.0
freq         37.0
Name: popularity_score, dtype: float64